In [ ]:
#from langchain.document_loaders import WebBaseLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from sentence_transformers import SentenceTransformer
#from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
#from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

In [ ]:
from bs4 import BeautifulSoup
import requests
from langchain_community.document_loaders import WebBaseLoader


url = "https://brainlox.com/courses/category/technical"
response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
soup = BeautifulSoup(response.text, "html.parser")


links = []
for a_tag in soup.find_all("a", href=True):
    link = a_tag["href"]
    if "brainlox.com" in link:  
        links.append(link)

print(f"Found {len(links)} links.")


all_docs = []
for link in links[:5]:  
    try:
        print(f"Loading: {link}")
        loader = WebBaseLoader(link)
        docs = loader.load()
        all_docs.extend(docs)
    except Exception as e:
        print(f"Failed to load {link}: {e}")


if all_docs:
    print("\nExtracted Content:\n", all_docs[0].page_content[:500])


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)


splits = text_splitter.split_documents(docs)

splits_text = [doc.page_content for doc in splits]


In [ ]:
model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
persist_directory = "docs/chroma/"


vectordb = Chroma.from_texts(
    texts=splits_text,
    embedding=model,
    persist_directory=persist_directory
)


print("Number of stored embeddings:", vectordb._collection.count())

In [ ]:
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline

# ✅ Load a local model using Transformers
#model_pipeline = pipeline("text-generation", model="facebook/opt-1.3b",max_new_tokens=200,)
model_pipeline = pipeline("text-generation", model="facebook/opt-350m", max_new_tokens=200)


# ✅ Define llm using HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=model_pipeline)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_kwargs={"k": 2})
)



In [ ]:

question = "What are major courses in python"
result = qa_chain.invoke({"query": question})

print(result["result"])